## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [1]:
!curl localhost:9200

{
  "name" : "6c4260c7e965",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "67cYp23EQU6XJYQRteKg-w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


"42f05b9372a9a4a470db3b52817899b99a76ee73"

## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [2]:
!pip install requests

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Note that you need to have the `requests` library:

```bash
pip install requests
```

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* **`index`**
* `put`
* `add`

In [4]:
from elasticsearch import Elasticsearch

In [7]:
es_client = Elasticsearch('http://localhost:9200')

if es_client.ping():
    print("Connected to Elasticsearch!")
else:
    print("Connection failed!")

Connected to Elasticsearch!


In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

try:
    es_client.indices.create(index=index_name, body=index_settings)
    print("Index created successfully!")
except Exception as e:
    print("Error creating index:", e)

Index created successfully!


In [12]:
from tqdm.auto import tqdm


for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/weslley/miniconda3/envs/venv_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 948/948 [00:31<00:00, 30.04it/s]


## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* **84.05**
* 74.05
* 64.05

Look at the `_score` field.

In [32]:
query = "How do I execute a command in a running docker container?"

In [31]:
from pprint import pprint 

search_query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'Dp73apABBcyVylldwRgM',
                    '_index': 'course-questions',
                    '_score': 84.050095,
                    '_source': {'course': 'machine-learning-zoomcamp',
                                'question': 'How do I debug a docker '
                                            'container?',
                                'section': '5. Deploying Machine Learning '
                                           'Models',
                                'text': 'Launch the container image in '
                                        'interactive mode and overriding the '
                                        'entrypoint, so that it starts a bash '
                                        'command.\n'
                                        'docker run -it --entrypoint bash '
                                        '<image>\n'
                                        'If th

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* **How do I copy files from a different folder into docker container’s working directory?**
* How do Lambda container images work?
* How can I annotate a graph?

In [34]:

search_query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'Dp73apABBcyVylldwRgM',
                    '_index': 'course-questions',
                    '_score': 84.050095,
                    '_source': {'course': 'machine-learning-zoomcamp',
                                'question': 'How do I debug a docker '
                                            'container?',
                                'section': '5. Deploying Machine Learning '
                                           'Models',
                                'text': 'Launch the container image in '
                                        'interactive mode and overriding the '
                                        'entrypoint, so that it starts a bash '
                                        'command.\n'
                                        'docker run -it --entrypoint bash '
                                        '<image>\n'
                                        'If th

In [42]:
response.body["hits"]["hits"][2].get("_source")["question"]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* **2462**

In [75]:
def elastic_search(query):

    search_query = {
        # "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [79]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT:
        {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n".strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [90]:
query = "How do I execute a command in a running docker container?"
docs = elastic_search(query)
prompt = build_prompt(query, docs)

In [91]:
len(prompt)

4476

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* **422**

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [82]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 2.3 MB/s eta 0:00:0000:0100:01


In [84]:
import tiktoken


encoding = tiktoken.encoding_for_model("gpt-4o")

In [96]:
encoding.encode(prompt)

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 558,
 309,
 7649,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 309,
 150339,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 309,
 31810,
 8099,
 734,
 309,
 1486,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 8,
 48,
 25,
 3253,
 621,
 357,
 5150,
 6291,
 591,
 922,
 2698,
 7342,
 316,
 62275,

In [93]:
len(encoding.encode(prompt))

1043

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [98]:
def llm(client, prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [99]:
def rag(client, query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(client, prompt)
    return answer

In [102]:
import dotenv

dotenv.load_dotenv()

True

In [103]:

from openai import OpenAI
client = OpenAI()

In [105]:
query = "How do I execute a command in a running docker container?"
response = rag(client, query)
print(response)

To execute a command in a running Docker container, follow these steps:

1. First, list all the running containers to find the container ID of the specific container:

    ```
    docker ps
    ```

2. Execute the command inside the running container using `docker exec`:

    ```
    docker exec -it <container-id> <command>
    ```

    For example, to start a bash session within the container:

    ```
    docker exec -it <container-id> bash
    ```

Replace `<container-id>` with the actual ID of your running container and `<command>` with the command you wish to execute inside the container.


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [114]:

def cost_of_response(prompt, response):
    input_tokens = len(encoding.encode(prompt))
    output_tokens = len(encoding.encode(response))
    
    cost_in = (0.005 * input_tokens) / 1000
    cost_out = (0.015 * output_tokens) / 1000
    
    return cost_in + cost_out

cost_of_response(prompt, response)

0.00718